In [5]:
dico_graphe = {}

In [6]:
def TrouveCommunaute(communautes, i):
    ''' Renvoie le numero de la communaute du noeud i'''
    
    return communautes[i]

In [7]:
def QuiDansCommunaute(communautes, C):
    ''' Renvoie une liste des noeuds appartenant la communauté C'''

    commu = []

    for n in range(len(communautes)):
        if communautes[n] == C:
            commu.append(n)
    
    return commu

In [8]:
def TrouveVoisins(dico, i):
    ''' Renvoie une liste des voisins du sommet i'''
    
    l_res = []
    v = dico[i]

    for j in v:
        l_res.append(j[0])

    return l_res

In [9]:
def AjoutDansCommunaute(communautes, i, C):
    ''' Ajoute i à la communauté C dans la liste communautés '''
    # communautes est une liste où chaque élément d'indice i est le numéro de la communauté de i

    communautes[i] = C
    return communautes

In [10]:
def MemeCommunaute(communautes, i, j):
    ''' Renvoie True si i et j sont dans la même communauté, False sinon'''

    if TrouveCommunaute(communautes, i) == TrouveCommunaute(communautes, j):
        return True
    return False

## Calcul du gain de modularité

In [11]:
def SommePoidsGraphe(dico):
    ''' Calcule la somme des poids du graphe (m) '''

    somme = 0
    cles = dico.keys()
    for c in cles:
        voisins = dico[c]
        for v in range(len(voisins)):
            somme += voisins[v][1]
    return somme//2

#m = SommePoidsGraphe(dico_graphe)   # appeler la fonction qu'une seule fois

In [12]:
def SommeInside(dico, communautes, C:int):
    ''' Calcule la somme des poids des arêtes dans la communauté C'''
    
    noeuds_de_C = QuiDansCommunaute(communautes, C)
    somme = 0
    
    for noeud in noeuds_de_C:
        voisins = dico[noeud]
        for v in voisins:
            if TrouveCommunaute(communautes, v[0]) == C:
                somme += v[1]
    
    return somme//2

In [13]:
def SommeTotale(dico, communautes, C:int):
    ''' Calcule la somme des poids des arêtes sortant de C (de tous les sommets appartenant à C)'''
    
    noeuds_de_C = QuiDansCommunaute(communautes, C)
    somme = 0
    
    for noeud in noeuds_de_C:
        voisins = dico[noeud]
        for v in voisins:
            if (v[0] not in noeuds_de_C) or (v[0] > noeud):        # vérifier qu'on a pas déjà compté cette arête => attention jsp
                somme += v[1]
    
    return somme

In [14]:
def Ki(dico, i):
    ''' Calcule la somme des poids des arêtes sortant (entrant) de i'''
    
    return len(dico[i])

In [15]:
def GainModularite(dico, communautes, m, i:int, j:int):
    ''' Calcule le gain de modularité si on ajoute i à la communauté de j'''

    C = TrouveCommunaute(communautes, j)
    S_in = SommeInside(dico, communautes, C)
    k_i_in = KiInside(dico, communautes, i)
    S_tot = SommeTotale(dico, communautes, C)
    k_i = Ki(dico, i)

    Q = (((S_in + 2*k_i_in)/(2*m)) - ((S_tot + k_i)/(2*m))**2) - ((S_in/(2*m)) - (S_tot/(2*m))**2 - (k_i/(2*m))**2)

    return Q

## Algorithme de Louvain

In [16]:
def Louvain1(dico):
    ''' Première partie de l'algorithme de Louvain '''

    flag_modif = True
    m = SommePoidsGraphe(dico)
    # Initialisation de la liste communautes : chaque noeud est dans une communauté différente 
    communautes = [k for k in range(len(dico))]

    while flag_modif == True:
        flag_modif = False          # aucune modification n'a encore été faite pour cette itération
        for i in range(len(communautes)):        # pour chaque noeud i
            max_gain = -1
            max_j = -1
            voisins_de_i = TrouveVoisins(dico, i)
            for j in voisins_de_i:      # pour chaque voisin de i
                gain_mod = GainModularite(dico, communautes, m, i, j)      # calcul du gain de modularité
                if i == 2 :
                    print('i:', i, 'j:', j, 'gain:', gain_mod)
                if gain_mod > max_gain:          # /!\ gérer l'égalité
                    max_gain = gain_mod
                    max_j = j
            if i == 1 :
                print('max_gain :  ', max_gain, 'max_j : ', max_j)
            #if max_gain > 0:        # /!\ strict ou pas ?
            flag_modif = True
            print("j'ajoute ", i, " à la communauté de ", max_j)
            communautes = AjoutDansCommunaute(communautes, max_j, TrouveCommunaute(communautes, i))       # ajout de i dans la communauté de j
            print(max_j, " est maintenant dans la communauté numéro ", communautes[max_j], " et i dans la numéro ", communautes[i])
            print("communautes : ", communautes)
    return communautes

In [17]:
def Louvain12(dico):
    ''' Première partie de l'algorithme de Louvain '''

    flag_modif = True
    m = SommePoidsGraphe(dico)
    # Initialisation de la liste communautes : chaque noeud est dans une communauté différente 
    communautes = [k for k in range(len(dico))]

    while flag_modif == True:
        flag_modif = False          # aucune modification n'a encore été faite pour cette itération
        for i in range(len(communautes)):        # pour chaque noeud i
            max_gain = -1
            max_j = -1
            voisins_de_i = TrouveVoisins(dico, i)
            k_i_in = '' # fonction en question
            for j in voisins_de_i:      # pour chaque voisin de i
                gain_mod = GainModularite(dico, communautes, m, i, j)      # calcul du gain de modularité
                if i == 2 :
                    print('i:', i, 'j:', j, 'gain:', gain_mod)
                if gain_mod > max_gain:          # /!\ gérer l'égalité
                    max_gain = gain_mod
                    max_j = j
            if i == 1 :
                print('max_gain :  ', max_gain, 'max_j : ', max_j)
            #if max_gain > 0:        # /!\ strict ou pas ?
            flag_modif = True
            print("j'ajoute ", i, " à la communauté de ", max_j)
            communautes = AjoutDansCommunaute(communautes, max_j, TrouveCommunaute(communautes, i))       # ajout de i dans la communauté de j
            print(max_j, " est maintenant dans la communauté numéro ", communautes[max_j], " et i dans la numéro ", communautes[i])
            print("communautes : ", communautes)
    return communautes

## Nouvel essai !

In [18]:
def Calcul_degre(dico, i):
    """ Retourne le degré du noeud i"""
    
    return len(dico[i])

In [19]:
def Calcul_ki_in(communautes, voisins_de_i, i, j):
    ''' Calcule la somme des poids des arêtes de i allant dans la communauté de j'''
    
    res = 0
    commu_de_j = communautes[j]

    for v in voisins_de_i:
        if communautes[v[0]] == commu_de_j:
            res += v[1]
    return res

#print(Calcul_ki_in([0, 1, 2, 1], [(1,1), (2,1), (3,1)], 0, 1))

In [20]:
def Calcul_S_tot():
    pass

In [62]:
def renommeCommu(commu):
    set_commu = list(set(commu))

    li_correspondance = [i for i in range(len(set_commu))]

    for i in range(len(commu)):
        for j in range(len(set_commu)):
            if commu[i] == set_commu[j]:
                commu[i] = li_correspondance[j]
    return commu

In [63]:
def Louvain(dico):
    
    flag_modif = True
    m = SommePoidsGraphe(dico)

    # Initialisation de la liste communautes : chaque noeud est dans une communauté différente 
    communautes = [k for k in range(len(dico))]

    while flag_modif == True:       # tant qu'on modifie encore les communautés
        flag_modif = False
        for noeud in range(len(communautes)):
            liste_voisins = TrouveVoisins(dico, noeud)
            max = -1
            max_v = -1
            
            liste_commu_voisins = []
            for vo in liste_voisins:
                liste_commu_voisins.append(communautes[vo])
                
            ki = len(liste_voisins)
            for v in liste_voisins:     # pour chaque voisin du noeud
                n_commu_v = QuiDansCommunaute(communautes, communautes[v])      # prendre la valeur [1]

                # Calcul de ki in
                ki_in = liste_commu_voisins.count(communautes[v])

                # Calcul de S tot
                S_tot = 0
                for ncv in n_commu_v:
                    if ncv != noeud :
                        S_tot += Calcul_degre(dico, ncv)
                
                # Calcul du gain
                gain = (2*ki_in) - (S_tot * ki)/m


#### tester positif
                if gain > max:      # si c'est un meilleur gain
                    max = gain
                    max_v = v
                
            if communautes[noeud] != communautes[max_v]:        # si on doit mettre le noeud dans une communauté différente
                AjoutDansCommunaute(communautes, noeud, communautes[max_v])     # on le met dans cette communauté en question
                flag_modif = True       # on a modifié
                #print('je viens de modifier')
                       

            #print(noeud, max_v, max)

    return renommeCommu(communautes)


            

In [41]:
liste_commu_voisins = [0, 1, 2, 2, 1, 0]

ki_in = liste_commu_voisins.count(2)

print(ki_in)

2


## Tests !

In [22]:
# Le graphe test contient 21 sommets et 34 arêtes

dico_test = {0:[(1,1)],
            1:[(0,1), (2,1)],
            2:[(1,1), (3,1), (4,1)],
            3:[(2,1), (4,1), (8,1)],
            4:[(2,1), (3,1), (5,1), (7,1)],
            5:[(4,1), (6,1), (17,1)],
            6:[(5,1), (7,1), (19,1)],
            7:[(4,1), (6,1), (9,1)],
            8:[(3,1)],
            9:[(7,1), (10,1), (11,1)],
            10:[(9,1), (11,1)],
            11:[(9,1), (10,1), (12,1)],
            12:[(11,1), (14,1), (17,1), (19,1)],
            13:[(14,1), (15,1), (16,1), (19,1)],
            14:[(12,1), (13,1), (15,1)],
            15:[(13,1), (14,1), (16,1), (17,1)],
            16:[(13,1), (15,1), (17,1), (18,1), (20,1)],
            17:[(5,1), (12,1), (15,1), (16,1), (18,1)],
            18:[(16,1), (17,1), (19,1), (20,1)],
            19:[(6,1), (12,1), (13,1), (18,1), (20,1)],
            20:[(16,1), (18,1), (19,1)]
            }

In [23]:
dico_test2 = {0:[(1,1), (2,1), (3,1)],
              1:[(0,1), (2,1), (3,1)],
              2:[(0,1), (1,1), (3,1)],
              3:[(0,1), (1,1), (2,1), (4,1)],
              4:[(5,1), (6,1), (7,1), (3,1)],
              5:[(4,1), (6,1), (7,1)],
              6:[(4,1), (5,1), (7,1)],
              7:[(4,1), (5,1), (6,1)],}

In [24]:
dico_SV = {0:[(2,1), (3,1), (4,1), (5,1)],
            1:[(2,1), (4,1), (7,1)],
            2:[(0,1), (1,1), (4,1), (5,1), (6,1),],
            3:[(0,1), (7,1)],
            4:[(0,1), (1,1), (2,1), (10,1)],
            5:[(0,1), (2,1), (7,1), (11,1)],
            6:[(2,1), (7,1), (11,1)],
            7:[(1,1), (3,1), (5,1), (6,1)],
            8:[(9,1), (10,1), (11,1), (14,1), (15,1)],
            9:[(8,1), (12,1), (14,1)],
            10:[(4,1), (8,1), (11,1), (12,1), (13,1), (14,1)],
            11:[(5,1), (6,1), (8,1), (10,1), (13,1)],
            12:[(9,1), (10,1)],
            13:[(10,1), (11,1)],
            14:[(8,1), (9,1), (10,1)],
            15:[(8,1)]}

In [61]:
res = Louvain(dico_SV)
print(res)

[4, 4, 4, 7, 4, 4, 7, 7, 12, 12, 12, 13, 12, 13, 12, 12]


In [60]:
renommeCommu(res)

[1, 1, 1, 3, 1, 1, 3, 3, 2, 2, 2, 0, 2, 0, 2, 2]

In [25]:
#print(GainModularite(dico_test, communautes, m, 6, 18))

In [26]:
commutest = [0, 1, 2, 0, 1, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
m = SommePoidsGraphe(dico_SV)
print(m)
print(GainModularite(dico_SV, commutest, m, 2, 1))

28


NameError: name 'KiInside' is not defined

In [ ]:
#print(Louvain1(dico_SV))

j'ajoute  0  à la communauté de  3
3  est maintenant dans la communauté numéro  0  et i dans la numéro  0
communautes :  [0, 1, 2, 0, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
max_gain :   -0.0076530612244897975 max_j :  4
j'ajoute  1  à la communauté de  4
4  est maintenant dans la communauté numéro  1  et i dans la numéro  1
communautes :  [0, 1, 2, 0, 1, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
i: 2 j: 0 gain: -0.01594387755102041
i: 2 j: 1 gain: -0.01913265306122449
i: 2 j: 4 gain: -0.01913265306122449
i: 2 j: 5 gain: -0.01275510204081633
i: 2 j: 6 gain: -0.009566326530612243
j'ajoute  2  à la communauté de  6
6  est maintenant dans la communauté numéro  2  et i dans la numéro  2
communautes :  [0, 1, 2, 0, 1, 5, 2, 7, 8, 9, 10, 11, 12, 13, 14, 15]
j'ajoute  3  à la communauté de  7
7  est maintenant dans la communauté numéro  0  et i dans la numéro  0
communautes :  [0, 1, 2, 0, 1, 5, 2, 0, 8, 9, 10, 11, 12, 13, 14, 15]
j'ajoute  4  à la communauté de  1
1  est maintenant dans la co

 est maintenant dans la communauté numéro  0  et i dans la numéro  0
communautes :  [0, 0, 0, 0, 0, 0, 0, 0, 0, 8, 0, 0, 8, 0, 8, 8]
j'ajoute  14  à la communauté de  9
9  est maintenant dans la communauté numéro  8  et i dans la numéro  8
communautes :  [0, 0, 0, 0, 0, 0, 0, 0, 0, 8, 0, 0, 8, 0, 8, 8]
j'ajoute  15  à la communauté de  8
8  est maintenant dans la communauté numéro  8  et i dans la numéro  8
communautes :  [0, 0, 0, 0, 0, 0, 0, 0, 8, 8, 0, 0, 8, 0, 8, 8]
j'ajoute  0  à la communauté de  2
2  est maintenant dans la communauté numéro  0  et i dans la numéro  0
communautes :  [0, 0, 0, 0, 0, 0, 0, 0, 8, 8, 0, 0, 8, 0, 8, 8]
max_gain :   0.06313775510204078 max_j :  2
j'ajoute  1  à la communauté de  2
2  est maintenant dans la communauté numéro  0  et i dans la numéro  0
communautes :  [0, 0, 0, 0, 0, 0, 0, 0, 8, 8, 0, 0, 8, 0, 8, 8]
i: 2 j: 0 gain: 0.10522959183673469
i: 2 j: 1 gain: 0.10522959183673469
i: 2 j: 4 gain: 0.10522959183673469
i: 2 j: 5 gain: 0.105229591836734

KeyboardInterrupt: 

In [ ]:
def RenvoieCommunautes(communautes):
    ''' A partir d'une liste où chaque élément d'indice i est le numéro de la communauté du noeud i,
        transforme en une autre liste ou chaque élément d'indice i est une liste des noeuds de la communauté i'''
    
    nb_commu = max(communautes)
    nv_communautes = [[] for k in range(nb_commu+1)]
    for i in range(len(communautes)):       # pour chaque noeud i
        nv_communautes[communautes[i]].append(i)

    return nv_communautes

In [ ]:
print(RenvoieCommunautes([0, 0, 0, 1, 1, 2, 2, 2, 2]))

[[0, 1, 2], [3, 4], [5, 6, 7, 8]]


In [ ]:
m = [0, 4, 6, 0, 2]
print(max(m))

6


In [ ]:
nv_communautes = [[] for k in range(5)]
print(nv_communautes)

[[], [], [], [], []]


In [ ]:
def Louvain(dico):
    
    m = SommePoidsGraphe(dico)

    # Initialisation de la liste communautes : chaque noeud est dans une communauté différente 
    communautes = [k for k in range(len(dico))]

    for noeud in communautes:
        liste_voisins = TrouveVoisins(dico, noeud)
        max = -1
        max_v = -1
        
        liste_commu_voisins = []
        for vo in liste_voisins:
            liste_commu_voisins.append(communautes[vo])
            
        ki = len(liste_voisins)
        for v in liste_voisins:
            ki_in = liste_commu_voisins.count(communautes[v])
            S_tot = 0
            n_commu_v = QuiDansCommunaute(communautes, communautes[v])      # prendre la valeur [1]
            for ncv in n_commu_v:
                if ncv != noeud :
                    S_tot += Calcul_degre(dico, ncv)

            gain = (2*ki_in) - (S_tot * ki)/m
            if gain > max:
                max = gain
                max_v = v
        
            #if noeud == 4:
                #print(v)
                #print('ki_in ', ki_in)
                #print('S-tot ', S_tot)

        AjoutDansCommunaute(communautes, noeud, communautes[max_v])

        print(noeud, max_v, max)


            